In [ ]:
from langchain_community.llms import Ollama
from langchain.embeddings import OllamaEmbeddings

In [ ]:
llm = Ollama(model = "llama3")
embedding = OllamaEmbeddings(model="nomic-embed-text")

# Document Loading

In [ ]:
from langchain.document_loaders import CSVLoader
filePath = "parking_spots.csv"

loader = CSVLoader(file_path=filePath)
data = loader.load()

# Document Extract

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
t_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

splits = t_splitter.split_documents(data)

# Document VectorStore

In [ ]:
# Method 1
from langchain.vectorstores import Chroma

presist_directory = "docs/chroma/"

vectordb = Chroma.from_documents(
    documents= splits, 
    embedding= embedding,
    persist_directory= presist_directory
)

In [ ]:
# Method 2
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding= embedding
).from_loaders([loader])

# Document Retreival

In [ ]:
question = "get me empty nomal parking near to lift"

In [ ]:
result = vectordb.similarity_search(question)
result

In [ ]:
from IPython.display import display, Markdown

indexResult = index.query(question, llm=llm)
display(Markdown(indexResult))

# Document QnA

In [ ]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

chat_history = []

memory = ConversationBufferMemory()
qa = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True
)

In [ ]:
question = "Looking for Empty parking"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for VIP Empty parking which is near to lift on floor 1"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for VIP Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for Normal Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))